In [1]:
import mne as mne
import numpy as np
import seaborn as sns
from scipy import stats
import pandas as pd
import os as os
import nolds as nolds

In [2]:
def hfd(a, k_max):

    L = []
    x = []
    N = a.size

    for k in range(1,k_max):
        Lk = 0
        for m in range(0,k):
            #we pregenerate all idxs
            idxs = np.arange(1,int(np.floor((N-m)/k)),dtype=np.int32)

            Lmk = np.sum(np.abs(a[m+idxs*k] - a[m+k*(idxs-1)]))
            Lmk = (Lmk*(N - 1)/(((N - m)/ k)* k)) / k
            Lk += Lmk


        L.append(np.log(Lk/(m+1)))
        x.append([np.log(1.0/ k), 1])

    p, r1, r2, s=np.linalg.lstsq(x, L)
    return p[0]

In [3]:
def hjorth(a):

    first_deriv = np.diff(a)
    second_deriv = np.diff(a,2)

    var_zero = np.mean(a ** 2)
    var_d1 = np.mean(first_deriv ** 2)
    var_d2 = np.mean(second_deriv ** 2)

    activity = var_zero
    morbidity = np.sqrt(var_d1 / var_zero)
    complexity = np.sqrt(var_d2 / var_d1) / morbidity

    return activity, morbidity, complexity

In [4]:
def pfd(a):

    diff = np.diff(a)
    # x[i] * x[i-1] for i in t0 -> tmax
    prod = diff[1:-1] * diff[0:-2]

    # Number of sign changes in derivative of the signal
    N_delta = np.sum(prod < 0)
    n = len(a)

    return np.log(n)/(np.log(n)+np.log(n/(n+0.4*N_delta)))

In [5]:
def do_univariate(rawdata):
    pctiles=rawdata.apply(stats.zscore).describe(percentiles=
                                                 np.concatenate([[0.001,0.01],np.arange(.1,1,.1),[0.99, 0.999]]))
    pctiles.loc['skew',:]=rawdata.apply(stats.zscore).skew().values
    pctiles.loc['kurtosis',:]=rawdata.apply(stats.zscore).kurtosis().values
    pctiles.loc['hyperskew',:]=rawdata.apply(stats.zscore).apply(stats.moment,moment=5).values
    pctiles.loc['nan%',:]=np.sum(rawdata.isnull().any()) /rawdata.shape[1]
    
    pctiles.loc['hurst',:]=rawdata.apply(nolds.hurst_rs)
    pctiles.loc['pfd',:]=rawdata.apply(stats.zscore).apply(pfd)
    #pctiles.loc['dfa',:]=rawdata.apply(nolds.dfa)
    
    pctiles.loc['hj2',:]=rawdata.apply(hjorth).iloc[1,:]
    pctiles.loc['hj3',:]=rawdata.apply(hjorth).iloc[2,:]

    pctiles.drop(['mean','min','max','std','count'],inplace=True)
    pctiles=pd.melt(pctiles.reset_index(), id_vars='index')
    pctiles['name'] = pctiles[['variable','index']].apply(lambda x: '_'.join(x), axis=1)
    pctiles.drop(columns=['variable','index'],inplace=True)
    pctiles=pctiles.set_index('name').T
    pctiles.rename(index={'value': thisfile},inplace=True)
    pctiles = pctiles.rename_axis(None, axis = 1)
    return pctiles

In [6]:
def do_base_bivariate(epochs):
    con_all, freqs, _, _, _ =mne.connectivity.spectral_connectivity(epochs,sfreq=info['sfreq'],verbose=False)
    return con_all, freqs

In [7]:
def do_maxmin_bivariate(epochs):
    tensor=list()
    for idx in np.arange(0,len(epochs.events)):
        temp, freqs, _, _, _ =mne.connectivity.spectral_connectivity(epochs[idx],sfreq=info['sfreq'],verbose=False)
        tensor.append(temp)
    con_max= np.stack(tensor, axis=3).max(axis=3)
    con_min= np.stack(tensor, axis=3).min(axis=3)
    return con_max, con_min

In [8]:
def do_fft(raw):
    normfft,f=mne.time_frequency.psd_multitaper(raw,normalization='full',verbose=False)
    return normfft,f

In [9]:
df_all=pd.DataFrame()

filespace=os.listdir('/media/jadhavlab/Data2/Volcanoes/predict-volcanic-eruptions-ingv-oe/train/')
for file in filespace:
    thisfile=file.split('.csv')[0]
    rawdata= pd.read_csv('/media/jadhavlab/Data2/Volcanoes/predict-volcanic-eruptions-ingv-oe/train/'+file)
    
    # clean bad columns
    rawdata.loc[:,(rawdata == 0).all(axis=0)]=np.nan
    
    mask = np.isnan(rawdata)
    masked_df = np.ma.masked_array(rawdata, mask)
    fill_value = pd.DataFrame({col: rawdata.mean(axis=1) for col in rawdata.columns})
    rawdata2 = masked_df.filled(fill_value)
    rawdata = pd.DataFrame(data=rawdata2, columns=rawdata.columns)

    # get univariate data
    pctiles=do_univariate(rawdata)
    
    # zscore
    data= stats.zscore(rawdata.values)
    # idxs of one matrix page
    relidx=np.tril_indices(data.shape[1],-1)
    
    # made mne array
    info = mne.create_info(ch_names=[str(x+1) for x in range(data.shape[1])],
                       sfreq=200., ch_types=['eeg']*data.shape[1],verbose=False)
    raw = mne.io.RawArray(data.T, info,verbose=False)
    events = mne.make_fixed_length_events(raw, duration=1.)
    epochs = mne.Epochs(raw, events, tmin=0.0, tmax=1.0,baseline=None,verbose=False)
    
    # do fft
    normfft,f=do_fft(raw);
    # do coherence of all pairs
    con_all, freqs=do_base_bivariate(epochs)
    # do min/max coherence
    con_max, con_min=do_maxmin_bivariate(epochs)
    
    sizeofmat=(relidx[0].shape[0],con_all.shape[2])
    
    reducedcon_all=np.zeros(sizeofmat)
    reducedcon_min=np.zeros(sizeofmat)
    reducedcon_max=np.zeros(sizeofmat)
    
    for i in range(con_all.shape[2]):
        temp_all= con_all[:,:,i]
        temp_min= con_min[:,:,i]
        temp_max= con_max[:,:,i]

        reducedcon_all[:,i]=temp_all[relidx]
        reducedcon_min[:,i]=temp_min[relidx]
        reducedcon_max[:,i]=temp_max[relidx]
        
    basepair=list(map('-'.join,zip(list(map(str,relidx[0]+1)),list(map(str,relidx[1]+1)))))
    basefreq=list(map(str,np.round(freqs).astype(int)))
    replbasepair=basepair*sizeofmat[1]
    replbasefreq=list(np.repeat(basefreq,sizeofmat[0]))
    
    columnnames_all=list(map('-all-'.join,zip(replbasepair,replbasefreq)))
    columnnames_min=list(map('-min-'.join,zip(replbasepair,replbasefreq)))
    columnnames_max=list(map('-max-'.join,zip(replbasepair,replbasefreq)))
    columnnames_fft=list(map(str,f.astype(float)))
    
    columnnames=columnnames_all+columnnames_min+columnnames_max+columnnames_fft
    
    df = pd.DataFrame(columns=columnnames)
    df.loc[0]=np.concatenate((reducedcon_all.flatten(order='F'),
               reducedcon_min.flatten(order='F'),
               reducedcon_max.flatten(order='F'),
               normfft.mean(axis=0)))
    df.rename(index={0: thisfile},inplace=True)
    df=df.join(pctiles)
    df_all=df_all.append(df)
    print(thisfile +' done')

1276176836 done
351509644 done
804951561 done
2042137471 done
1725501178 done
2007716403 done
84070532 done
1147795366 done
323620345 done
1726460483 done
924684934 done
921899048 done
781524342 done
2132437848 done
1519559429 done
2005284845 done
209988785 done
1409167039 done
304573399 done
582268580 done
807819960 done
1111781387 done
2091434754 done
1567108572 done
1207757591 done
1847339053 done
180442009 done
424555504 done
1872501987 done
1432155265 done
1068054446 done
797240914 done
585908475 done
1544540742 done
1265760121 done
2041042967 done
861257947 done
1804779174 done
412373157 done
1141054211 done
512339405 done
1871098842 done
162722008 done
973336622 done
1074244722 done
694445719 done
1151918483 done
1946443471 done
164890155 done
616719593 done
815391918 done
1878766314 done
1886769185 done
1016242026 done
1315506288 done
2082273024 done
1945782318 done


/home/jadhavlab/anaconda3/lib/python3.7/site-packages/mne/connectivity/spectral.py:91: RuntimeWarning: invalid value encountered in true_divide
  self.con_scores[con_idx] = np.abs(csd_mean) / np.sqrt(psd_xx * psd_yy)


1848578834 done
1943060459 done
615891154 done
1408645616 done
777577982 done
1089685571 done
1772142112 done
1999816124 done
997431578 done
621938105 done
1160333827 done
270906952 done
1577179301 done
1379040576 done
847334954 done
84813922 done
1002609771 done
503921694 done
1646467816 done
1693633195 done
992018586 done
1450261744 done
1249783920 done
214183849 done
1859470905 done
1207122523 done
1606405966 done
834484140 done
903626058 done
586537093 done
118578176 done
63715701 done
1810962014 done
633652522 done
1421646167 done
1262119935 done
2041531962 done
1920256938 done
984224331 done
101616763 done
1381523950 done
27870347 done
288067094 done
450769375 done
516212556 done
647980942 done
442489148 done
1414856943 done
219469142 done
184996575 done
864547424 done
873082841 done
1821339602 done
1964855818 done
1746134419 done
2055241392 done
474206182 done
765529516 done
930971928 done
1845995648 done


/home/jadhavlab/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:594: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


191737931 done
39008491 done
150706632 done
570052095 done
546732136 done
1070060959 done
740985437 done
573514854 done
348486495 done
293321906 done
1031098571 done
272101269 done
625038024 done
927854113 done
1102017109 done
2130505106 done
869119721 done
367264529 done
1938473432 done
524875318 done
2036967215 done
1629106107 done
2102366851 done
1376574453 done
185435567 done
1780106336 done
438232368 done
556592677 done
621992624 done
1003520023 done
451860145 done
1936159281 done
51938320 done
1838577064 done
1092333463 done
1772905707 done
2030114890 done
1393369578 done
2129738903 done
676899080 done
1806903214 done
1586315342 done
1858752827 done
799740613 done
1192354436 done
1746205662 done
176230486 done
2109398679 done
952401993 done
2042918322 done
1262173496 done
1570560259 done
1042857837 done
1305823886 done
281177678 done
982455895 done
369856026 done
339846368 done
582324215 done
705698742 done
1762541807 done
1710136076 done
697387932 done
2031334949 done
949734355 

...

1177828269 done
422675999 done
1567324269 done
2015589007 done
1792016264 done
25177348 done
841450874 done
1427376683 done


/home/jadhavlab/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:594: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


2052316250 done


/home/jadhavlab/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:594: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


546825827 done
2133808067 done
718462915 done
1358099181 done
692506393 done
762505241 done
786817871 done
964004526 done
1948878169 done
1139540990 done
691651457 done
1577280457 done
300148624 done
597690122 done
623083505 done
1002789083 done
1615710791 done
1139907613 done
1300979382 done
1523753816 done
427149670 done
629137203 done
754268958 done


In [10]:
df_all

,2-1-all-6,3-1-all-6,3-2-all-6,4-1-all-6,4-2-all-6,4-3-all-6,5-1-all-6,5-2-all-6,5-3-all-6,5-4-all-6,...,sensor_10_99%,sensor_10_99.9%,sensor_10_skew,sensor_10_kurtosis,sensor_10_hyperskew,sensor_10_nan%,sensor_10_hurst,sensor_10_pfd,sensor_10_hj2,sensor_10_hj3
1276176836,0.033980,0.074766,0.021637,0.082512,0.044106,0.033530,0.011595,0.098947,0.015463,0.030035,...,2.697789,4.997371,0.044201,4.116455,0.765226,0.0,0.697005,1.005663,0.228305,1.824473
351509644,0.008751,0.017197,0.035110,0.005255,0.052367,0.083114,0.068274,0.090429,0.022815,0.035073,...,2.387579,3.634612,-0.049398,1.350269,-3.476323,0.0,0.493367,1.003158,0.138372,2.206347
804951561,0.292896,0.109651,0.292461,0.013414,0.280615,0.056864,0.008349,0.249973,0.044896,0.043644,...,2.737902,5.339124,0.098601,4.058365,4.077087,0.0,0.508492,1.008802,0.187894,4.399597
2042137471,0.019240,0.048984,0.034675,0.039838,0.058540,0.043867,0.067687,0.076968,0.065418,0.065219,...,2.581070,4.320717,0.007809,2.283654,-0.352362,0.0,0.385049,1.002767,0.138871,1.837450
1725501178,0.026066,0.070268,0.009590,0.047863,0.066168,0.051748,0.039233,0.082149,0.051595,0.019259,...,2.525949,4.005425,-0.002659,1.479222,-0.315787,0.0,0.585221,1.004521,0.203490,1.946645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300979382,0.720478,0.207152,0.022411,0.192692,0.055057,0.057323,1.000000,0.720478,0.207152,0.192692,...,2.531331,3.877604,0.080157,1.149752,1.768196,0.0,0.600445,1.004288,0.171010,2.204689
1523753816,0.016436,0.046097,0.059142,0.084538,0.054263,0.018358,0.011025,0.025171,0.015291,0.032625,...,2.821647,6.341718,-0.211955,15.619445,-18.951720,0.0,0.739810,1.008125,0.235424,2.663305
427149670,0.036914,0.044625,0.008257,0.055030,0.010282,0.014662,0.241488,0.632866,0.176734,0.276213,...,2.436471,3.362923,0.032233,0.397437,-0.045648,0.0,0.909643,1.008571,0.140131,5.387428
629137203,0.016393,0.068225,0.015026,0.095140,0.036159,0.059262,0.057522,0.159460,0.013588,0.055833,...,2.408480,3.391281,0.032724,0.210067,0.536649,0.0,0.331011,1.002158,0.118753,1.714874


In [11]:
df_all.to_csv('/media/jadhavlab/Data2/Volcanoes/train.csv')
